### Proyecto Final Grupal HENRY

**ML_3_algoritmo**  
Prueba del sistema de puntuación final

**Autores:**

**Camino Federico**  
**Londero Walter**  
**Pizarro Hernan**  
**Urteaga Facundo**  
**Veron Cintia**   

**Resumen:** Prueba del sistema de puntuación final, previo a la implementación directa en otro tipo de soporte

1.  **Carga de librerías y datos**
2.  **Pre procesamiento de dataframes**
3.  **Simulación interfaz usuario**
4.  **Simulación interfaz inmobiliaria**
5.  **Funciones**
6.  **Prueba didáctica de funciones**
7.  **Guardo DF para analytics**

#### 1. Carga de librerías y datos

In [1]:
# Carga de librerías

import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') # Para evitar los molestos avisos.
import math

In [2]:
# Carga de archivos

df_locales = pd.read_csv("locales_LA.csv")
df_fc_1 = pd.read_csv("fc_results_1.csv")
df_fc_2 = pd.read_csv("fc_results_2.csv")
df_fc_3 = pd.read_csv("fc_results_3.csv")

# Defino la calidad de la predicción de cada grupo

df_fc_1['forecast'] = 'débil'
df_fc_2['forecast'] = 'media'
df_fc_3['forecast'] = 'fuerte'

In [3]:
df_locales

,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,Hours_day,Hours_night,general_category,res_asian,res_latin,res_euro,res_fast,res_vegan,religion,recreation,bienestar,avg_rating_correction
0,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,['Korean restaurant'],4.4,18,54,11,Food,1,0,0,0,0,0,0,0,4.565217
1,"El Zacatecas Sports Bar, 12017 Venice Blvd., L...",0x80c2ba5b61bb2415:0x217207f872e3db5e,34.006111,-118.429722,['Sports bar' 'Bar'],4.0,18,63,28,Food,0,0,0,0,0,0,3,0,4.230769
2,"Marina Bar & Restaurant, Tom Bradley Internati...",0x80c2b12739335d6b:0xea7f9dd4f3c9f585,33.943646,-118.408984,['Bar & grill' 'American restaurant' 'Bar'],4.4,5,77,0,Food,0,0,0,1,0,0,0,0,NaN
3,"PIZZA ROW RECORDS, 433 E 7th St, Los Angeles, ...",0x80c2c7726b53b8e5:0xad288d11654f5e48,34.041406,-118.247477,['Pizza restaurant'],4.3,34,95,18,Food,0,0,0,1,0,0,0,0,4.777778
4,"La Superior Panaderia, 2001 W 8th St, Los Ange...",0x80c2c798ad41d859:0x8823cd86d2abd5b3,34.054542,-118.276768,['Mexican restaurant'],4.3,30,84,21,Food,0,1,0,0,0,0,0,0,4.484848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2809,"Oishi Teriyaki, 1830 W 8th St B, Los Angeles, ...",0x80c2c7a27259a639:0xbd8b00242c57dc6e,34.053447,-118.275408,['Japanese restaurant' 'Asian restaurant'],4.4,88,79,0,Food,1,0,0,0,0,0,0,0,NaN
2810,"PRMO'S ANTOJITOS MEXICANOS, 2235 E 1st St, Los...",0x80c2c6060f244b5d:0xba79c65be0d3ec63,34.044611,-118.211358,['Mexican restaurant'],4.8,76,0,0,Food,0,1,0,0,0,0,0,0,NaN
2811,"Pollo Mundial, 800 S Alvarado St, Los Angeles,...",0x80c2c798cb3c3c05:0x53df87852638dfc8,34.054527,-118.277487,['Delivery Restaurant' 'Chicken restaurant' 'M...,4.2,65,63,0,Food,0,1,0,1,0,0,0,0,NaN
2812,"Secret Garden, 1925 W Olympic Blvd, Los Angele...",0x80c2c79850acba0b:0x6d4492c188d75810,34.051381,-118.278202,['Korean restaurant' 'Buffet restaurant' 'Rest...,3.7,15,21,0,Food,1,0,0,0,0,0,0,0,NaN


#### 2. Pre procesamiento de dataframes

In [4]:
# Crear una lista con los DataFrames que quieres concatenar
df_fc_1 = df_fc_1.rename(columns={'predicted_three_ratings': 'next_three_forecast'})
df_fc_2 = df_fc_2.rename(columns={'predicted_three_ratings': 'next_three_forecast'})

dataframes = [df_fc_1, df_fc_2, df_fc_3]

# Concatenar los DataFrames a lo largo del eje 0 (filas)
sum_df = pd.concat(dataframes, ignore_index=True)

In [5]:
# Función para convertir strings a listas
def convertir_a_lista(cadena):
    return ast.literal_eval(cadena)

# Convertir las columnas de strings a listas
sum_df['last_three_ratings'] = sum_df['last_three_ratings'].apply(convertir_a_lista)
sum_df['next_three_forecast'] = sum_df['next_three_forecast'].apply(convertir_a_lista)

In [6]:
# Función para calcular el promedio de los 6 valores
def calcular_promedio(row):
    combined_values = row['last_three_ratings'] + row['next_three_forecast']
    return sum(combined_values) / len(combined_values)

# Aplicar la función para calcular el promedio y definir la tendencia
sum_df['average_six_values'] = sum_df.apply(calcular_promedio, axis=1)

# Definir la tendencia basada en el promedio calculado
def definir_tendencia(promedio):
    if promedio > 4.0:
        return 'Alta'
    elif promedio > 3.0:
        return 'Media'
    else:
        return 'Baja'

sum_df['tendencia'] = sum_df['average_six_values'].apply(definir_tendencia)

In [7]:
sum_df["tendencia"].value_counts()

tendencia
Alta     554
Media    160
Baja      49
Name: count, dtype: int64

In [8]:
# Realizar join entre df_locales y forecast_results_df_1 por gmap_id, conservando todas las filas de df_locales
df_locales_ml = pd.merge(df_locales, sum_df, on='gmap_id', how='left')

# Definir las condiciones para evaluar
conditions = [
    (df_locales_ml['Hours_day'] != 0) & (df_locales_ml['Hours_night'] == 0),  # Solo Hours_day tiene valor distinto de 0
    (df_locales_ml['Hours_day'] == 0) & (df_locales_ml['Hours_night'] != 0),  # Solo Hours_night tiene valor distinto de 0
    (df_locales_ml['Hours_day'] != 0) & (df_locales_ml['Hours_night'] != 0)   # Ambos tienen valores distintos de 0
]

# Definir los valores a asignar para cada condición
values = [0, 1, 2]

# Crear la nueva columna utilizando np.select
df_locales_ml['time_category'] = np.select(conditions, values, default=np.nan)

# Mostrar el resultado del join
df_locales_ml.head()

,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,Hours_day,Hours_night,general_category,...,religion,recreation,bienestar,avg_rating_correction,last_three_ratings,next_three_forecast,forecast,average_six_values,tendencia,time_category
0,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,['Korean restaurant'],4.4,18,54,11,Food,...,0,0,0,4.565217,"[5.0, 5.0, 5.0]","[4.81, 4.81, 4.81]",débil,4.905000,Alta,2.0
1,"El Zacatecas Sports Bar, 12017 Venice Blvd., L...",0x80c2ba5b61bb2415:0x217207f872e3db5e,34.006111,-118.429722,['Sports bar' 'Bar'],4.0,18,63,28,Food,...,0,3,0,4.230769,"[2.5, 5.0, 5.0]","[4.73, 4.73, 4.73]",débil,4.448333,Alta,2.0
2,"Marina Bar & Restaurant, Tom Bradley Internati...",0x80c2b12739335d6b:0xea7f9dd4f3c9f585,33.943646,-118.408984,['Bar & grill' 'American restaurant' 'Bar'],4.4,5,77,0,Food,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,"PIZZA ROW RECORDS, 433 E 7th St, Los Angeles, ...",0x80c2c7726b53b8e5:0xad288d11654f5e48,34.041406,-118.247477,['Pizza restaurant'],4.3,34,95,18,Food,...,0,0,0,4.777778,"[4.5, 5.0, 5.0]","[4.96, 4.96, 4.96]",débil,4.896667,Alta,2.0
4,"La Superior Panaderia, 2001 W 8th St, Los Ange...",0x80c2c798ad41d859:0x8823cd86d2abd5b3,34.054542,-118.276768,['Mexican restaurant'],4.3,30,84,21,Food,...,0,0,0,4.484848,"[4.5, 4.0, 4.0]","[4.53, 4.53, 4.53]",débil,4.348333,Alta,2.0


In [9]:
df_locales_ml["time_category"].value_counts()

time_category
0.0    1133
2.0     716
Name: count, dtype: int64

In [10]:
# Lista de columnas para las cuales se desea obtener los value_counts
columns_to_count = ['res_asian', 'res_latin', 'res_euro', 'res_fast', 'res_vegan', 'religion', 'recreation', 'bienestar']

# Iterar sobre cada columna y mostrar los value_counts
for column in columns_to_count:
    print(f"Value counts for {column}:")
    print(df_locales_ml[column].value_counts())
    print("\n")  # Línea en blanco para separar las salidas de cada columna

Value counts for res_asian:
res_asian
0    2377
1     437
Name: count, dtype: int64


Value counts for res_latin:
res_latin
0    2373
1     441
Name: count, dtype: int64


Value counts for res_euro:
res_euro
0    2661
1     153
Name: count, dtype: int64


Value counts for res_fast:
res_fast
0    2194
1     620
Name: count, dtype: int64


Value counts for res_vegan:
res_vegan
0    2694
1     120
Name: count, dtype: int64


Value counts for religion:
religion
0    2511
1     201
2      42
3      36
5      13
4      11
Name: count, dtype: int64


Value counts for recreation:
recreation
0    2506
2     145
1     137
3      26
Name: count, dtype: int64


Value counts for bienestar:
bienestar
0    2086
1     728
Name: count, dtype: int64




#### 3. Simulación de interfaz de usuario

In [11]:
# Simulación interfaz usuario

# Códigos:

# Weight:
#   0) Nada
#   1) Poco
#   2) Normal
#   3) Mucho

# Restaurant:
#   1) Comida asiática 
#   2) Comida latinoamericana
#   3) Comida europea
#   4) Comida norteamericana y fast food
#   5) Comida vegana y vegetariana

# Religion:
#   1) Protestanismo 
#   2) Catolisismo
#   3) Judaísmo
#   4) Budismo
#   5) Islam

# Recreacion:
#   1) Arte, música, teatro, cine, bibliotecas
#   2) Clubes nocturnos
#   3) Salas de Bowling, pool, dardos, casino

# Cronotipo:
#   0) Diruno
#   1) Nocturno
#   2) Indiferente/ambos

# Preferencias del usuario 1
user_preferences_1 = {
    "Restaurant": {
        "weight": 3,
        "sub_category": 4
    },
    "Religion": {
        "weight": 1,
        "sub_category": 4
    },
    "Recreation": {
        "weight": 2,
        "sub_category": 1
    },
    "Cronotipo": 2
}

user_preferences_2 = {
    "Restaurant": {
        "weight": 2,
        "sub_category": 1
    },
    "Religion": {
        "weight": 3,
        "sub_category": 1
    },
    "Recreation": {
        "weight": 1,
        "sub_category": 3
    },
    "Cronotipo": 0
}

user_preferences_3 = {
    "Restaurant": {
        "weight": 0,
        "sub_category": 4
    },
    "Religion": {
        "weight": 1,
        "sub_category": 3
    },
    "Recreation": {
        "weight": 3,
        "sub_category": 2
    },
    "Cronotipo": 1
}

In [12]:
df_locales_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2814 entries, 0 to 2813
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   address                2814 non-null   object 
 1   gmap_id                2814 non-null   object 
 2   latitude               2814 non-null   float64
 3   longitude              2814 non-null   float64
 4   category               2814 non-null   object 
 5   avg_rating             2814 non-null   float64
 6   num_of_reviews         2814 non-null   int64  
 7   Hours_day              2814 non-null   int64  
 8   Hours_night            2814 non-null   int64  
 9   general_category       2814 non-null   object 
 10  res_asian              2814 non-null   int64  
 11  res_latin              2814 non-null   int64  
 12  res_euro               2814 non-null   int64  
 13  res_fast               2814 non-null   int64  
 14  res_vegan              2814 non-null   int64  
 15  reli

#### 4. Simulación de interfaz inmobiliaria

In [13]:
# Coordenada de ejemplo: lat/long 34.058092	-118.292130

# Suponemos area de 1 km cuadrado. Es decir, sumar y restar el equivalente de 500 metros a cada coordenada
# En lat 500 m aprox 0.0045 grados
# En long 500 m aprox 0.0054 grados

# Inmueble 1:
lat_1 = 34.058092
lon_1 = -118.292130

# Inmueble 2:
lat_2 = 34.032025
lon_2 = -118.244198

# Inmueble 3:
lat_3 = 34.047421
lon_3 = -118.256777

# Inmueble 4:
lat_4 = 34.027630
lon_4 = -118.266843

# Inmueble 5:
lat_5 = 34.015112
lon_5 = -118.251910

# Inmueble 6:
lat_5 = 33.993401
lon_5 = -118.278310

#### 5. Funciones

In [14]:
# calcular_distancia: Función accesoria que se utiliza para calcular la distancia lineal entre dos puntos

def calcular_distancia(lat_1, long_1, lat_2, long_2):
    # Constantes para convertir grados a metros
    METROS_POR_GRADO_LAT = 111320  # Aprox. para todas las latitudes
    METROS_POR_GRADO_LONG = 111320 * math.cos(math.radians(lat_1))  # Varía según la latitud

    # Diferencias en grados
    delta_lat = lat_2 - lat_1
    delta_long = long_2 - long_1

    # Convertir diferencias en grados a diferencias en metros
    delta_lat_metros = delta_lat * METROS_POR_GRADO_LAT
    delta_long_metros = delta_long * METROS_POR_GRADO_LONG

    # Calcular distancia euclidiana
    distancia_metros = math.sqrt(delta_lat_metros**2 + delta_long_metros**2)
    
    # Redondear a la centena más cercana
    distancia_redondeada = round(distancia_metros, -2)
    
    return distancia_redondeada

In [15]:
# filtrar_locales: Define los locales a tener en cuenta a partir de la vecindad del inmueble y las preferencias del usuario

def filtrar_locales(df_locales, lat_inm, lon_inm, km, user_pref):
    
    df_locales_copy = df_locales.copy()

    # Función para determinar la categoría
    def determinar_categoria(row):
        if any(row[['res_asian', 'res_latin', 'res_euro', 'res_fast', 'res_vegan']] != 0):
            return 'res'
        elif row['religion'] != 0:
            return 'rel'
        elif row['recreation'] != 0:
            return 'rec'
        elif row['bienestar'] != 0:
            return 'bien'
        else:
            return np.nan

    # Crear la nueva columna 'ml_category'
    df_locales_copy['ml_category'] = df_locales_copy.apply(determinar_categoria, axis=1)
    
    # Filtrar el DataFrame según el área determinada
    df_locales_copy = df_locales_copy[
    (df_locales_copy['latitude'] >= (lat_inm - 0.0045 * km * 2)) & 
    (df_locales_copy['latitude'] <= (lat_inm + 0.0045 * km * 2)) &
    (df_locales_copy['longitude'] >= (lon_inm - 0.0054 * km * 2)) & 
    (df_locales_copy['longitude'] <= (lon_inm + 0.0054 * km * 2))
    ]
    
    # Filtrar el DataFrame a partir de las preferencias del usuario

    # Filtrar según el peso
    if user_pref['Restaurant']['weight'] == 0:
        df_locales_copy = df_locales_copy[
            (df_locales_copy['res_asian'] != 1) &
            (df_locales_copy['res_latin'] != 1) &
            (df_locales_copy['res_euro'] != 1) &
            (df_locales_copy['res_fast'] != 1) &
            (df_locales_copy['res_vegan'] != 1)
        ]
    if user_pref['Religion']['weight'] == 0:
        df_locales_copy = df_locales_copy[df_locales_copy['religion'] != 1]
    if user_pref['Recreation']['weight'] == 0:
        df_locales_copy = df_locales_copy[df_locales_copy['recreation'] != 1]

    # Filtrar según la sub-categoría para "Restaurant"
    restaurant_sub_category = user_pref['Restaurant']['sub_category']
    if restaurant_sub_category == 1:
        df_locales_copy = df_locales_copy[
            (df_locales_copy['res_asian'] == 1) |
            ((df_locales_copy['res_asian'] == 0) & 
            (df_locales_copy['res_latin'] == 0) & 
            (df_locales_copy['res_euro'] == 0) & 
            (df_locales_copy['res_fast'] == 0) & 
            (df_locales_copy['res_vegan'] == 0))
        ]
    elif restaurant_sub_category == 2:
        df_locales_copy = df_locales_copy[
            (df_locales_copy['res_latin'] == 1) |
            ((df_locales_copy['res_asian'] == 0) & 
            (df_locales_copy['res_latin'] == 0) & 
            (df_locales_copy['res_euro'] == 0) & 
            (df_locales_copy['res_fast'] == 0) & 
            (df_locales_copy['res_vegan'] == 0))
        ]
    elif restaurant_sub_category == 3:
        df_locales_copy = df_locales_copy[
            (df_locales_copy['res_euro'] == 1) |
            ((df_locales_copy['res_asian'] == 0) & 
            (df_locales_copy['res_latin'] == 0) & 
            (df_locales_copy['res_euro'] == 0) & 
            (df_locales_copy['res_fast'] == 0) & 
            (df_locales_copy['res_vegan'] == 0))
        ]
    elif restaurant_sub_category == 4:
        df_locales_copy = df_locales_copy[
            (df_locales_copy['res_fast'] == 1) |
            ((df_locales_copy['res_asian'] == 0) & 
            (df_locales_copy['res_latin'] == 0) & 
            (df_locales_copy['res_euro'] == 0) & 
            (df_locales_copy['res_fast'] == 0) & 
            (df_locales_copy['res_vegan'] == 0))
        ]
    elif restaurant_sub_category == 5:
        df_locales_copy = df_locales_copy[
            (df_locales_copy['res_vegan'] == 1) |
            ((df_locales_copy['res_asian'] == 0) & 
            (df_locales_copy['res_latin'] == 0) & 
            (df_locales_copy['res_euro'] == 0) & 
            (df_locales_copy['res_fast'] == 0) & 
            (df_locales_copy['res_vegan'] == 0))
        ]

    # Filtrar según la sub-categoría para "Religion"
    religion_sub_category = user_pref['Religion']['sub_category']
    df_locales_copy = df_locales_copy[
        (df_locales_copy['religion'] == religion_sub_category) |
        (df_locales_copy['religion'] == 0)
    ]

    # Filtrar según la sub-categoría para "Recreation"
    recreation_sub_category = user_pref['Recreation']['sub_category']
    df_locales_copy = df_locales_copy[
        (df_locales_copy['recreation'] == recreation_sub_category) |
        (df_locales_copy['recreation'] == 0)
    ]

    # Dejo solo columnas de interes
    
    # Añadir columna "distance" al DataFrame
    df_locales_copy['distance'] = df_locales_copy.apply(
        lambda row: calcular_distancia(row['latitude'], row['longitude'], lat_inm, lon_inm),
        axis=1
    )
    df_filtrado = df_locales_copy[['address','gmap_id', 'latitude', 'longitude','avg_rating',
       'num_of_reviews','religion', 'recreation', 'bienestar', 'avg_rating_correction','forecast', 'tendencia', 'time_category', 'distance','ml_category']]

    return df_filtrado

In [16]:
# calcular_puntuacion: Se ingresa el df filtrado por la función anterior y las preferencias de usuario. Devuelve la puntuación del inmueble dado y cadenas de texto
# con la distancia y puntuación de los mejores locales de cada categoría

def calcular_puntuacion(df_prueba, user_pref):
    total_score = 0
    score_res = 0
    score_rel = 0
    score_rec = 0
    score_bien = 0

    best_restaurant = ""
    best_bienestar = ""
    best_recreation = ""
    best_religion = ""

    df = df_prueba.copy()

    # Restaurantes (bienestar, recreation, religion son 0)
    restaurants = df[(df['bienestar'] == 0) & (df['recreation'] == 0) & (df['religion'] == 0)]
    if len(restaurants) > 0:
        if len(restaurants) == 1:
            score_res = 4
        elif len(restaurants) in [2, 3]:
            score_res = 4.3
        else:
            score_res = 4.5
        
        max_avg_rating = max(restaurants['avg_rating'].max(), restaurants['avg_rating_correction'].max())
        if max_avg_rating > 4.5:
            score_res += 0.3
        
        # Encontrar el restaurante con el max_avg_rating más alto
        best_restaurant_idx = restaurants.loc[
            (restaurants['avg_rating'] == max_avg_rating) |
            (restaurants['avg_rating_correction'] == max_avg_rating)
        ].index[0]
        
        if df.at[best_restaurant_idx, 'forecast'] == 'fuerte':
            if df.at[best_restaurant_idx, 'tendencia'] == 'Alta':
                score_res += 0.2
        elif df.at[best_restaurant_idx, 'tendencia'] == 'Alta':
            score_res += 0.1

        # Guardar el mejor restaurante
        best_restaurant = df.at[best_restaurant_idx, 'address']
        best_restaurant_name = best_restaurant.split(',')[0]
        best_restaurant_rating = max(
        df.at[best_restaurant_idx, 'avg_rating'], 
        df.at[best_restaurant_idx, 'avg_rating_correction'] if pd.notna(df.at[best_restaurant_idx, 'avg_rating_correction']) else -np.inf
        )
        best_restaurant_rating = round(best_restaurant_rating, 2)
        best_restaurant_dist = df.at[best_restaurant_idx, 'distance']
        
    
    # Bienestar (bienestar es 1)
    bienestar = df[df['bienestar'] == 1]
    if len(bienestar) > 0:
        if len(bienestar) == 1:
            score_bien = 1.5
        elif len(bienestar) in [2, 3]:
            score_bien = 2
        else:
            score_bien = 2.5
        
        max_avg_rating = max(bienestar['avg_rating'].max(), bienestar['avg_rating_correction'].max())
        if max_avg_rating > 4.5:
            score_bien += 0.3
        
        best_bienestar_idx = bienestar.loc[
            (bienestar['avg_rating'] == max_avg_rating) |
            (bienestar['avg_rating_correction'] == max_avg_rating)
        ].index[0]
        
        if df.at[best_bienestar_idx, 'forecast'] == 'fuerte':
            if df.at[best_bienestar_idx, 'tendencia'] == 'Alta':
                score_bien += 0.2
        elif df.at[best_bienestar_idx, 'tendencia'] == 'Alta':
            score_bien += 0.1
        
        # Guardar el mejor bienestar
        best_bienestar = df.at[best_bienestar_idx, 'address']
        best_bienestar_name = best_bienestar.split(',')[0]
        best_bienestar_rating = max(
        df.at[best_bienestar_idx, 'avg_rating'], 
        df.at[best_bienestar_idx, 'avg_rating_correction'] if pd.notna(df.at[best_bienestar_idx, 'avg_rating_correction']) else -np.inf
        )
        best_bienestar_rating = round(best_bienestar_rating, 2)
        best_bienestar_dist = df.at[best_bienestar_idx, 'distance']
    
    # Recreación (recreation es distinto de 0)
    recreation = df[df['recreation'] != 0]
    if len(recreation) > 0:
        if len(recreation) == 1:
            score_rec = 0.5
        elif len(recreation) in [2, 3]:
            score_rec = 0.6
        else:
            score_rec = 0.7
        
        max_avg_rating = max(recreation['avg_rating'].max(), recreation['avg_rating_correction'].max())
        if max_avg_rating > 4.5:
            score_rec += 0.1
        
        best_recreation_idx = recreation.loc[
            (recreation['avg_rating'] == max_avg_rating) |
            (recreation['avg_rating_correction'] == max_avg_rating)
        ].index[0]
        
        if df.at[best_recreation_idx, 'forecast'] == 'fuerte':
            if df.at[best_recreation_idx, 'tendencia'] == 'Alta':
                score_rec += 0.2
        elif df.at[best_recreation_idx, 'tendencia'] == 'Alta':
            score_rec += 0.1
        
        # Guardar el mejor recreación
        best_recreation = df.at[best_recreation_idx, 'address']
        best_recreation_name = best_recreation.split(',')[0]
        best_recreation_rating = max(
        df.at[best_recreation_idx, 'avg_rating'], 
        df.at[best_recreation_idx, 'avg_rating_correction'] if pd.notna(df.at[best_recreation_idx, 'avg_rating_correction']) else -np.inf
        )
        best_recreation_rating = round(best_recreation_rating, 2)
        best_recreation_dist = df.at[best_recreation_idx, 'distance']
    
    # Religión (religion es distinto de 0)
    religion = df[df['religion'] != 0]
    if len(religion) > 0:
        if len(religion) == 1:
            score_rel = 0.5
        elif len(religion) in [2, 3]:
            score_rel = 0.6
        else:
            score_rel = 0.7
        
        max_avg_rating = max(religion['avg_rating'].max(), religion['avg_rating_correction'].max())
        if max_avg_rating > 4.5:
            score_rel += 0.1
        
        best_religion_idx = religion.loc[
            (religion['avg_rating'] == max_avg_rating) |
            (religion['avg_rating_correction'] == max_avg_rating)
        ].index[0]
        
        if df.at[best_religion_idx, 'forecast'] == 'fuerte':
            if df.at[best_religion_idx, 'tendencia'] == 'Alta':
                score_rel += 0.2
        elif df.at[best_religion_idx, 'tendencia'] == 'Alta':
            score_rel += 0.1
        
        # Guardar el mejor religión
        best_religion = df.at[best_religion_idx, 'address']
        best_religion_name = best_religion.split(',')[0]
        best_religion_rating = max(
        df.at[best_religion_idx, 'avg_rating'], 
        df.at[best_religion_idx, 'avg_rating_correction'] if pd.notna(df.at[best_religion_idx, 'avg_rating_correction']) else -np.inf
        )
        best_religion_rating = round(best_religion_rating, 2)
        best_religion_dist = df.at[best_religion_idx, 'distance']


    # Ponderación final del puntaje:
    p_res = user_pref['Restaurant']['weight'] # Puede ser 0, 1, 2 o 3
    p_rec = user_pref['Recreation']['weight']
    p_rel = user_pref['Religion']['weight']

    def ajustar_score(score, ponderacion):
        if ponderacion == 1:
            return score * 0.8
        elif ponderacion == 3:
            return score + (5 - score) * 0.1
        return score

    score_res = ajustar_score(score_res, p_res)
    score_rec = ajustar_score(score_rec, p_rec)
    score_rel = ajustar_score(score_rel, p_rel)
    
    total_score = score_res + score_rel + score_bien + score_rec

    if p_res == 0:
        total_score = (5 * total_score) / 10
        
    elif p_rec == 0:
        total_score = (9 * total_score) / 10
    
    elif p_rel == 0:
        total_score = (9 * total_score) / 10
    
    messages = []
    if best_restaurant:
        messages.append(f"Tienes un restaurante llamado '{best_restaurant_name}' a {best_restaurant_dist} metros de esta ubicación con una valoración de {best_restaurant_rating}")
    if best_bienestar:
        messages.append(f"Tienes un '{best_bienestar_name}' a {best_bienestar_dist} metros de esta ubicación con una valoración de {best_bienestar_rating}")
    if best_recreation:
        messages.append(f"Te podrás divertir en '{best_recreation_name}' a {best_recreation_dist} metros de esta ubicación con una valoración de {best_recreation_rating}")
    if best_religion:
        messages.append(f"Tienes un '{best_religion_name}' a {best_religion_dist} metros de esta ubicación con una valoración de {best_religion_rating}")
    
    return total_score, score_res, score_rel, score_rec, score_bien, messages


#### 6. Prueba didáctica de funciones

In [17]:
# Prueba de funciones con combinaciones de ubicaciones y usuarios

latitudes = [lat_1, lat_2, lat_3, lat_4, lat_5]
longitudes = [lon_1, lon_2, lon_3, lon_4, lon_5]
user_preferences_list = [user_preferences_1, user_preferences_2, user_preferences_3]

# Aplicar las funciones a cada combinación de (lat, lon) y preferencias de usuario
for i, (lat, lon) in enumerate(zip(latitudes, longitudes)):
    for j, user_pref in enumerate(user_preferences_list):
        # Filtrar locales
        df_filtered = filtrar_locales(df_locales_ml, lat, lon, 1, user_pref)
        
        # Calcular el puntaje
        total_score2, score_res2, score_rel2, score_rec2, score_bien2, mensajes = calcular_puntuacion(df_filtered, user_pref)
        
        # Mostrar los resultados en el formato deseado
        print(f"lat_{i+1} lon_{i+1} user_pref_{j+1} scor2: {total_score2:.1f} score_res: {score_res2:.1f} score_rel: {score_rel2:.1f} score_rec: {score_rec2:.1f} score_bien: {score_bien2:.1f}")

lat_1 lon_1 user_pref_1 scor2: 8.9 score_res: 4.9 score_rel: 0.5 score_rec: 0.7 score_bien: 2.8
lat_1 lon_1 user_pref_2 scor2: 9.3 score_res: 4.8 score_rel: 1.2 score_rec: 0.5 score_bien: 2.8
lat_1 lon_1 user_pref_3 scor2: 2.1 score_res: 0.0 score_rel: 0.0 score_rec: 1.3 score_bien: 2.8
lat_2 lon_2 user_pref_1 scor2: 8.2 score_res: 4.9 score_rel: 0.0 score_rec: 0.5 score_bien: 2.8
lat_2 lon_2 user_pref_2 scor2: 8.6 score_res: 4.9 score_rel: 0.5 score_rec: 0.4 score_bien: 2.8
lat_2 lon_2 user_pref_3 scor2: 2.0 score_res: 0.0 score_rel: 0.0 score_rec: 1.1 score_bien: 2.8
lat_3 lon_3 user_pref_1 scor2: 8.4 score_res: 4.8 score_rel: 0.0 score_rec: 0.8 score_bien: 2.8
lat_3 lon_3 user_pref_2 scor2: 9.2 score_res: 4.8 score_rel: 1.0 score_rec: 0.6 score_bien: 2.8
lat_3 lon_3 user_pref_3 scor2: 2.2 score_res: 0.0 score_rel: 0.5 score_rec: 1.2 score_bien: 2.8
lat_4 lon_4 user_pref_1 scor2: 8.0 score_res: 4.5 score_rel: 0.0 score_rec: 0.7 score_bien: 2.8
lat_4 lon_4 user_pref_2 scor2: 8.8 score

In [18]:
# Prueba de mensajes

df_filtered = filtrar_locales(df_locales_ml, lat_2, lon_2, 1, user_preferences_3)
total_score, score_res, score_rel, score_rec, score_bien, mensaje = calcular_puntuacion(df_filtered, user_preferences_1)
print(mensaje[1])

Te podrás divertir en 'Flying Embers Taproom' a 900.0 metros de esta ubicación con una valoración de 4.6


#### 7. Guardo dataframe para el departamento de analytics

In [19]:
# Función para agregar una fila al principio del DataFrame
def agregar_fila(df, lat, lon):
    new_row = pd.DataFrame({
        'address': ["inmueble"],
        'gmap_id': [np.nan],
        'latitude': [lat],
        'longitude': [lon],
        'avg_rating': [np.nan],
        'avg_rating_correction': [np.nan],
        'distance': [np.nan],
        'ml_category': [np.nan]
    })
    
    return new_row


In [20]:
# Crear un DataFrame vacío con las mismas columnas que df_filtered
df_final = pd.DataFrame(columns=['address', 'gmap_id', 'latitude', 'longitude', 'avg_rating',
                                 'avg_rating_correction', 'distance', 'ml_category',
                                 'rating_final', 'score_inm', 'user_pref', 'inmueble'])

latitudes = [lat_1, lat_2, lat_3, lat_4, lat_5]
longitudes = [lon_1, lon_2, lon_3, lon_4, lon_5]
user_preferences_list = [user_preferences_1, user_preferences_2, user_preferences_3]

# Aplicar las funciones a cada combinación de (lat, lon) y preferencias de usuario
for i, (lat, lon) in enumerate(zip(latitudes, longitudes)):
    for j, user_pref in enumerate(user_preferences_list):
        # Filtrar locales
        df_filtered = filtrar_locales(df_locales_ml, lat, lon, 1, user_pref)
        
        # Calcular el puntaje
        total_score, score_res, score_rel, score_rec, score_bien, mensajes = calcular_puntuacion(df_filtered, user_pref)

        df_filtered = df_filtered[['address', 'gmap_id', 'latitude', 'longitude', 'avg_rating',
                                   'avg_rating_correction', 'distance', 'ml_category']]
        
        # Concatenar la nueva fila con el DataFrame existente
        new_row = agregar_fila(df_filtered, lat, lon)
        df_filtered = pd.concat([new_row, df_filtered], ignore_index=True)

        # Crear la nueva columna 'rating_final'
        df_filtered['rating_final'] = np.where(
            df_filtered['avg_rating_correction'].notna(), 
            df_filtered['avg_rating_correction'], 
            df_filtered['avg_rating']
        )

        # Añadir las columnas 'score_inm' y 'user_pref'
        df_filtered['score_inm'] = total_score
        df_filtered['user_pref'] = f'user_pref_{j+1}'
        df_filtered['inmueble'] = f'inmueble_{i+1}'

        # Concatenar df_filtered con df_final
        df_final = pd.concat([df_final, df_filtered], ignore_index=True)


In [21]:
import pandas as pd
import numpy as np

# Crear un DataFrame vacío con las mismas columnas que df_filtered
df_final = pd.DataFrame(columns=['address', 'gmap_id', 'latitude', 'longitude', 'avg_rating',
                                 'avg_rating_correction', 'distance', 'ml_category',
                                 'rating_final', 'score_inm', 'user_pref', 'inmueble'])

# Inicializar df_anexo como un DataFrame vacío
df_anexo = pd.DataFrame(columns=['inmueble', 'user_pref', 'user_res_p', 'user_res_c', 
                                 'user_rel_p', 'user_rel_c', 'user_rec_p', 'user_rec_c', 'mens'])

latitudes = [lat_1, lat_2, lat_3, lat_4, lat_5]
longitudes = [lon_1, lon_2, lon_3, lon_4, lon_5]
user_preferences_list = [user_preferences_1, user_preferences_2, user_preferences_3]

# Aplicar las funciones a cada combinación de (lat, lon) y preferencias de usuario
for i, (lat, lon) in enumerate(zip(latitudes, longitudes)):
    for j, user_pref in enumerate(user_preferences_list):
        # Filtrar locales
        df_filtered = filtrar_locales(df_locales_ml, lat, lon, 1, user_pref)
        
        # Calcular el puntaje
        total_score, score_res, score_rel, score_rec, score_bien, mensajes = calcular_puntuacion(df_filtered, user_pref)

        df_filtered = df_filtered[['address', 'gmap_id', 'latitude', 'longitude', 'avg_rating',
                                   'avg_rating_correction', 'distance', 'ml_category']]
        
        # Concatenar la nueva fila con el DataFrame existente
        new_row = agregar_fila(df_filtered, lat, lon)
        df_filtered = pd.concat([new_row, df_filtered], ignore_index=True)

        # Crear la nueva columna 'rating_final'
        df_filtered['rating_final'] = np.where(
            df_filtered['avg_rating_correction'].notna(), 
            df_filtered['avg_rating_correction'], 
            df_filtered['avg_rating']
        )

        # Añadir las columnas 'score_inm' y 'user_pref'
        df_filtered['score_inm'] = total_score
        df_filtered['user_pref'] = f'user_pref_{j+1}'
        df_filtered['inmueble'] = f'inmueble_{i+1}'
        df_filtered['inmuser_id'] = f'u{j+1}i{i+1}'

        # Concatenar df_filtered con df_final
        df_final = pd.concat([df_final, df_filtered], ignore_index=True)

        # Crear el DataFrame anexo
        anexo_row = {
            'inmueble': f'inmueble_{i+1}',
            'user_pref': f'user_pref_{j+1}',
            'inmuser_id': f'u{j+1}i{i+1}',
            'user_res_p': user_pref['Restaurant']['weight'],
            'user_res_c': user_pref['Restaurant']['sub_category'],
            'user_rel_p': user_pref['Religion']['weight'],
            'user_rel_c': user_pref['Religion']['sub_category'],
            'user_rec_p': user_pref['Recreation']['weight'],
            'user_rec_c': user_pref['Recreation']['sub_category'],
            'mens': ', '.join(mensajes)  # Unir los mensajes en un solo string
        }
        
        # Convertir anexo_row en un DataFrame temporal y concatenarlo con df_anexo
        df_anexo = pd.concat([df_anexo, pd.DataFrame([anexo_row])], ignore_index=True)


In [22]:
df_final.head()

,address,gmap_id,latitude,longitude,avg_rating,avg_rating_correction,distance,ml_category,rating_final,score_inm,user_pref,inmueble,inmuser_id
0,inmueble,NaN,34.058092,-118.292130,NaN,NaN,NaN,NaN,NaN,8.89,user_pref_1,inmueble_1,u1i1
1,"847 S Vermont Ave Parking, 3038W W 8th St, Los...",0x80c2c77897fffa8b:0x5c2904deb5734e19,34.056428,-118.291801,3.0,NaN,200.0,bien,3.000000,8.89,user_pref_1,inmueble_1,u1i1
2,"mirae medical clinic, 2675 W Olympic Blvd #202...",0x80c2c783a0bd3031:0x117bf83e39fbd829,34.052670,-118.290634,3.5,NaN,600.0,bien,3.500000,8.89,user_pref_1,inmueble_1,u1i1
3,"Wi Spa Customer Parking, 2701W 7th St, Los Ang...",0x80c2c77667d9201b:0x4f286f8fc3438bca,34.060011,-118.283239,3.0,NaN,800.0,bien,3.000000,8.89,user_pref_1,inmueble_1,u1i1
4,"Subway, 3323 Wilshire Blvd Ground Floor, Los A...",0x80c2c77c6a3968f1:0xee3b615679d47cdf,34.061971,-118.295066,3.8,3.920455,500.0,res,3.920455,8.89,user_pref_1,inmueble_1,u1i1


In [23]:
df_anexo

,inmueble,user_pref,user_res_p,user_res_c,user_rel_p,user_rel_c,user_rec_p,user_rec_c,mens,inmuser_id
0,inmueble_1,user_pref_1,3,4,1,4,2,1,Tienes un restaurante llamado 'Fire Chicken' a...,u1i1
1,inmueble_1,user_pref_2,2,1,3,1,1,3,Tienes un restaurante llamado 'SYTK Sullungtan...,u2i1
2,inmueble_1,user_pref_3,0,4,1,3,3,2,Tienes un '조아라 내과 / Ahra Cho MD' a 600.0 metro...,u3i1
3,inmueble_2,user_pref_1,3,4,1,4,2,1,Tienes un restaurante llamado 'Es Todo Vegan S...,u1i2
4,inmueble_2,user_pref_2,2,1,3,1,1,3,Tienes un restaurante llamado 'Hayato' a 300.0...,u2i2
5,inmueble_2,user_pref_3,0,4,1,3,3,2,Tienes un 'BodyByNiisign' a 1100.0 metros de e...,u3i2
6,inmueble_3,user_pref_1,3,4,1,4,2,1,Tienes un restaurante llamado 'Danny Boy's Fam...,u1i3
7,inmueble_3,user_pref_2,2,1,3,1,1,3,Tienes un restaurante llamado 'Poketendo' a 10...,u2i3
8,inmueble_3,user_pref_3,0,4,1,3,3,2,Tienes un 'Westlawn Garage' a 400.0 metros de ...,u3i3
9,inmueble_4,user_pref_1,3,4,1,4,2,1,Tienes un restaurante llamado 'JT Cafe' a 900....,u1i4


In [24]:
# Exportar ambos dataframes como csv

df_final.to_csv('df_final.csv', index=False)
df_anexo.to_csv('df_anexo.csv', index=False)